# Tensors in torch

In [1]:
import torch
import numpy as np

In [2]:
torch.manual_seed(0)
np.random.seed(0)

## 1. Creating tensors 

### 1.1 Basics

Tensors can be created using specific commands:

In [34]:
#torch.empty(2, 2)

In [4]:
torch.zeros(2, 2)

tensor([[0., 0.],
        [0., 0.]])

In [5]:
torch.rand(1,3,1, dtype=torch.double)

tensor([[[0.9701],
         [0.7078],
         [0.4594]]], dtype=torch.float64)

Or directly from data:

In [6]:
torch.Tensor([[1,2,3]])

tensor([[1., 2., 3.]])

### 1.2 `torch.Tensor` attributes

Each `torch.Tensor` object has three attributtes:
* `torch.dtype`
* `torch.device`
* `torch.layout`

#### `torch.dtype`

A `torch.dtype` is an object that represents the type of a `torch.Tensor`. There are **nine** different dtypes in torch:

| dtype  | alternative type |
|---|:---|
| torch.float32  | torch.float  |
| torch.float64  | torch.double  |
| torch.float16  | torch.half  |
| torch.uint8  | -  |
| torch.int8  | -  |
| torch.int32  | torch.int  |
| torch.int64  | torch.long  |
| torch.int16  | torch.short  |
| torch.bool  | -  |

#### `torch.device`

A `torch.device` is an object representing the device where the `torch.Tensor` is or will be allocated. Admits `cpu` or `cuda`, and the number to reference this resource. A `torch.device` admits the following notations:

```python
torch.device('cuda', 1)
torch.device('cuda:1')
```

#### `torch.layout`

A `torch.layout` is an object that represents the memory layout of a `torch.Tensor`: `torch.strided` for dense matrices or `torch.sparse_coo` for sparse matrices.

For a dense matrices, `torch.Tensor.stride()` is a list. In the k-th position of this list we can find the number of elements that must be skipped to jump to the next element the k-th dimension. For example:

In [7]:
x = torch.ones(3,2)
x

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [8]:
x.stride()

(2, 1)

This means: to jump to the next element in the first dimension (bi-dimensional arrays), we need to skip two elements. In the second dimension, only one element must be skipped to reach the following element.

In [9]:
x = torch.zeros(3,2,2)
x

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [10]:
x.stride()

(4, 2, 1)

### 1.3 Reusing tensor properies

In torch we can create new tensors reusing properties of other tensors.

##### `.new_*` methods

Let's create a new tensor with dtype `torch.uint8`.

In [11]:
x = torch.zeros(1, 2, dtype=torch.uint8)
x

tensor([[0, 0]], dtype=torch.uint8)

Now, let's create a new tensor with the properties of x (in this case, dtype=`torch.uint8`)

In [12]:
x.new_ones(2,3) # new shape is required

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.uint8)

##### `torch.*_like` functions

This kind of functions take an existing tensor, and create a new one mantaining its properties (including the shape).

In [13]:
x = torch.zeros(1, 2, dtype=torch.double)
torch.rand_like(x)

tensor([[0.9207, 0.6450]], dtype=torch.float64)

## 2. Working with `torch.Tensor`'s

One of the most frequent operations you need when working with tensors is getting its size:

In [14]:
x.size()

torch.Size([1, 2])

* A `torch.Size` objects behaves like a tuple

### 2.1 Operations

In [15]:
x, y = torch.zeros(1, 2), torch.ones(1, 2)
x, y

(tensor([[0., 0.]]), tensor([[1., 1.]]))

Common operations have been overloaded for `torch.Tensor` objects. For example, this sum

In [16]:
x + y

tensor([[1., 1.]])

is using this operation:

In [17]:
torch.add(x, y)

tensor([[1., 1.]])

However, this torch-native operation have some interesting options. For example, we can specify the name of a variable to keep this result in memory:

In [18]:
added_tensors = torch.empty(1,1) # if the shape doesn't match, added_tensors will be resaped automatically
torch.add(x, y, out=added_tensors)

tensor([[1., 1.]])

#### in-place operations

In torch, any operation or method post-fixed with a `_` is an in-place operation.

In [19]:
x

tensor([[0., 0.]])

In [20]:
x.add_(y)

tensor([[1., 1.]])

In [21]:
x

tensor([[1., 1.]])

Now transposing ...

In [22]:
x.t()

tensor([[1.],
        [1.]])

In [23]:
x

tensor([[1., 1.]])

In [24]:
x.t_()

tensor([[1.],
        [1.]])

In [25]:
x

tensor([[1.],
        [1.]])

These are the basics about operations. If you want to explore all the possibilities, here you can find the official guide: https://pytorch.org/docs/stable/torch.html

### 2.2 Slicing

Like in numpy!

In [26]:
torch.ones(3, 1)[:2,0]

tensor([1., 1.])

### 2.3 Reshaping

In torch, tensor resaping is done using the `.view()` method

In [27]:
x = torch.ones(2, 3)
x.view(6)

tensor([1., 1., 1., 1., 1., 1.])

In [28]:
x.view(3,2)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

To infer a dimension size, we can use `-1`

In [29]:
x = torch.ones(12)
x.reshape(2, -1, 3).size()

torch.Size([2, 2, 3])

## 3. Interaction with other formats or libraries

#### from one-sized `torch.Tensor` to python int

With the `.item()` method we can get the value of the tensor:

In [30]:
torch.ones(1).item()

1.0

#### from `torch.Tensor` to `numpy` arrays

In [31]:
torch.ones(2,3).numpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

#### from `numpy` to `torch.Tensor`

In [32]:
x = np.ones((2,3), dtype=np.float32)
torch.from_numpy(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

#### to keep in mind

* numpy only works on CPUs
* All the numeric `torch.Tensor` living on the CPU support converting to NumPy and back (this is not supported for CharTensor).

## Moving `torch.Tensor`'s between devices

We can move `torch.Tensor`'s between devices using the `.to` method.

In [33]:
if torch.cuda.is_available():
    my_device = torch.device('cuda', 0)
    
    # x is created on the CPU, and moved to the GPU
    x = torch.ones(2, 2, dtype=torch.float)
    x.to(my_device) # x.to("cuda") also works
    
    # y is created on the GPU
    y = torch.ones(2, 2, dtype=torch.float, device=my_device)
    
    # sums on the GPU
    z = torch.add(x, y)
    
    z.to("cpu") # z.to(torch.device('cpu')) also works ;)

## Reference:

https://pytorch.org/docs/stable/torch.html

https://pytorch.org/docs/stable/tensor_attributes.html

https://pytorch.org/docs/stable/tensors.html

https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html